# Install

In [ ]:
!pip3 install swifter

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install pip==20.0.2 pororo fairseq==0.10.2

In [ ]:
!pip uninstall torch torchvision

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu   # cpuonly

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import torch
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.13.1+cpu
cuda version: None
cudnn version:None


# Data Load

In [10]:
import os
os.chdir('/content/drive/MyDrive/Mbti_LexicalTendency/src')

import config as cfg

In [11]:
from pprint import pprint
import pandas as pd
from pshmodule.utils import filemanager as fm

In [14]:
# nf = pd.read_excel(cfg.nf, engine="openpyxl")
st = pd.read_excel(cfg.st, engine="openpyxl", header=1)

In [15]:
st.head()

,speaker,uttterance
0,화자1,그러면 제일 먼저 이거부터 해보자.
1,화자1,독심술 초능력이 생겼는데 모든 생각 읽기 대 거짓말하면 죽는 병에 걸리기 너는 뭐 ...
2,화자2,난 거짓말하면 죽는 병에 걸리기.
3,화자1,나도 나도 나도.
4,화자1,거짓말하면 죽는 병에 걸리기


In [17]:
st.dropna(inplace=True)

In [18]:
st.shape

(1125, 2)

In [19]:
# nf = nf[['speaker', 'uttterance']]
st = st[['speaker', 'uttterance']]

In [20]:
# nf.rename(columns={'uttterance':'utterance'}, inplace=True)
st.rename(columns={'uttterance':'utterance'}, inplace=True)

In [21]:
st.head()

,speaker,utterance
0,화자1,그러면 제일 먼저 이거부터 해보자.
1,화자1,독심술 초능력이 생겼는데 모든 생각 읽기 대 거짓말하면 죽는 병에 걸리기 너는 뭐 ...
2,화자2,난 거짓말하면 죽는 병에 걸리기.
3,화자1,나도 나도 나도.
4,화자1,거짓말하면 죽는 병에 걸리기


# Pororo

In [22]:
import swifter
from pororo import Pororo

In [23]:
Pororo.available_models("dp")
dp = Pororo(task="dep_parse", lang="ko")

In [24]:
# nf['pororo'] = nf.utterance.swifter.apply(lambda x: dp(x))
st['pororo'] = st.utterance.swifter.apply(lambda x: dp(x))

Pandas Apply:   0%|          | 0/1125 [00:00<?, ?it/s]

# ST

다른 mbti 나오면 배열 처리 ㄱㄱ

In [29]:
# window size
data_list = [2, 3, 4, 5, 6]

for num in data_list:
  exec(f'result_{str(num)} = []')
  for i in st.iterrows():
    for j in range(num-1, len(i[1]['pororo'])):
      temp = []
      temp.append(i[1]['utterance'])
      if num == 6:
        # token6
        temp.append(i[1]['pororo'][j-5][1])
        temp.append(i[1]['pororo'][j-5][3])
        temp.append(i[1]['pororo'][j-4][1])
        temp.append(i[1]['pororo'][j-4][3])
        temp.append(i[1]['pororo'][j-3][1])
        temp.append(i[1]['pororo'][j-3][3])
        temp.append(i[1]['pororo'][j-2][1])
        temp.append(i[1]['pororo'][j-2][3])

      elif num == 5:
        # token5
        temp.append(i[1]['pororo'][j-4][1])
        temp.append(i[1]['pororo'][j-4][3])        
        temp.append(i[1]['pororo'][j-3][1])
        temp.append(i[1]['pororo'][j-3][3])
        temp.append(i[1]['pororo'][j-2][1])
        temp.append(i[1]['pororo'][j-2][3])
        
      elif num == 4:
        # token4
        temp.append(i[1]['pororo'][j-3][1])
        temp.append(i[1]['pororo'][j-3][3])
        temp.append(i[1]['pororo'][j-2][1])
        temp.append(i[1]['pororo'][j-2][3])

      elif num == 3:
        # token3
        temp.append(i[1]['pororo'][j-2][1])
        temp.append(i[1]['pororo'][j-2][3])

      # token2
      temp.append(i[1]['pororo'][j-1][1])
      temp.append(i[1]['pororo'][j-1][3])      
      # token1
      temp.append(i[1]['pororo'][j][1])
      temp.append(i[1]['pororo'][j][3])
      
      exec(f'result_{str(num)}.append({temp})')

In [30]:
print(f"result_2 : {result_2[:5]}")
print(f"result_3 : {result_3[:5]}")
print(f"result_4 : {result_4[:5]}")
print(f"result_5 : {result_5[:5]}")
print(f"result_6 : {result_6[:5]}")

result_2 : [['그러면 제일 먼저 이거부터 해보자.', '그러면', 'AP', '제일', 'AP'], ['그러면 제일 먼저 이거부터 해보자.', '제일', 'AP', '먼저', 'AP'], ['그러면 제일 먼저 이거부터 해보자.', '먼저', 'AP', '이거부터', 'NP_AJT'], ['그러면 제일 먼저 이거부터 해보자.', '이거부터', 'NP_AJT', '해보자.', 'VP'], ['독심술 초능력이 생겼는데 모든 생각 읽기 대 거짓말하면 죽는 병에 걸리기 너는 뭐 할 거야?', '독심술', 'NP', '초능력이', 'NP_SBJ']]
result_3 : [['그러면 제일 먼저 이거부터 해보자.', '그러면', 'AP', '제일', 'AP', '먼저', 'AP'], ['그러면 제일 먼저 이거부터 해보자.', '제일', 'AP', '먼저', 'AP', '이거부터', 'NP_AJT'], ['그러면 제일 먼저 이거부터 해보자.', '먼저', 'AP', '이거부터', 'NP_AJT', '해보자.', 'VP'], ['독심술 초능력이 생겼는데 모든 생각 읽기 대 거짓말하면 죽는 병에 걸리기 너는 뭐 할 거야?', '독심술', 'NP', '초능력이', 'NP_SBJ', '생겼는데', 'VP'], ['독심술 초능력이 생겼는데 모든 생각 읽기 대 거짓말하면 죽는 병에 걸리기 너는 뭐 할 거야?', '초능력이', 'NP_SBJ', '생겼는데', 'VP', '모든', 'DP']]
result_4 : [['그러면 제일 먼저 이거부터 해보자.', '그러면', 'AP', '제일', 'AP', '먼저', 'AP', '이거부터', 'NP_AJT'], ['그러면 제일 먼저 이거부터 해보자.', '제일', 'AP', '먼저', 'AP', '이거부터', 'NP_AJT', '해보자.', 'VP'], ['독심술 초능력이 생겼는데 모든 생각 읽기 대 거짓말하면 죽는 병에 걸리기 너는 뭐 할 거야?', '독심술', 'NP', '초능력이', 'NP_SBJ', '생겼는데', 'VP', '모

In [31]:
frames = {
  'st2': "pd.DataFrame(result_2, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag']])",
  'st3': "pd.DataFrame(result_3, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag', 'token3', 'token3_tag']])",
  'st4': "pd.DataFrame(result_4, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag', 'token3', 'token3_tag', 'token4', 'token4_tag']])",
  'st5': "pd.DataFrame(result_5, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag', 'token3', 'token3_tag', 'token4', 'token4_tag', 'token5', 'token5_tag']])",
  'st6': "pd.DataFrame(result_6, columns=[['utterance', 'token1', 'token1_tag', 'token2', 'token2_tag', 'token3', 'token3_tag', 'token4', 'token4_tag', 'token5', 'token5_tag', 'token6', 'token6_tag']])",
}

for name, df in frames.items():
  exec(f'{name} = {df}')

In [ ]:
st6.head()

# Save

In [38]:
import os
os.chdir('/content/drive/MyDrive/Mbti_LexicalTendency/src')

In [39]:
import config as cfg

In [41]:
with pd.ExcelWriter(cfg.st_save) as writer:
    st.to_excel(writer, sheet_name = 'origin')
    st2.to_excel(writer, sheet_name = 'ngram2')
    st3.to_excel(writer, sheet_name = 'ngram3')
    st4.to_excel(writer, sheet_name = 'ngram4')
    st5.to_excel(writer, sheet_name = 'ngram5')
    st6.to_excel(writer, sheet_name = 'ngram6')